In [87]:
# import libraries
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import re

In [153]:
# gets: url=the url of the prodoct, headers=the thing needed to get amazon html
# return: nothing
# the function requests the prodocts html page from amazon, 
# and scraps all the info we want, 
# then the list of info back

def single_prodocts_handle(url, headers):
    if(url == None):
        print('single_prodocts_handle got url=None')
        return None
    page = requests.get(url, headers=headers)
    soup1 = BeautifulSoup(page.content, 'html.parser')
    soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
    
    prodoct_list = []
    
    # scrap info
    title = None
    title = soup2.find(id='productTitle')
    if title is not None:
        title = title.get_text().strip()
    prodoct_list.append(title)
    #print(title)
    
    # price
    price_text = None
    #price = soup2.find_all("span", {"class": "a-offscreen"})
    #apexPriceToPay
    price = soup2.find_all("span", {"class": "apexPriceToPay"})
    for ext in price:
        price_text = ext.find("span", {"class": "a-offscreen"}).getText().strip()
        #print(place_name)
    #PriceToPay
    price = soup2.find_all("span", {"class": "priceToPay"})
    for ext in price:
        price_text = ext.find("span", {"class": "a-offscreen"}).getText().strip()
        #print(place_name)
    prodoct_list.append(price_text)
    
    # year
    year = None
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    for td in soup2.find_all("td", {"class": "a-size-base prodDetAttrValue"}):
        if any(ext in td.text for ext in months):
            year = td.text.strip()
            #print(year)
            break
    prodoct_list.append(year)
    
    # all the other traits
    traits = ['po-brand','po-model_name','po-display.size','po-hard_disk.size','po-cpu_model.family','po-ram_memory.installed_size','po-operating_system','po-graphics_coprocessor']
    for trait in traits:
        place_trait = soup2.find_all("tr", {"class": trait})
        if place_trait:
            for ext in place_trait:
                place_name = ext.find("td", {"class": "a-span9"})
                if(place_name is not None):
                    place_name = place_name.getText().strip()
                #print(trait,': ', place_name)
                prodoct_list.append(place_name)
        else:
            prodoct_list.append(None)
            
    return prodoct_list


In [166]:
def search_page_handler(url, headers):
    #if(url == None):
    #    print('search_page_handler got url=None')
    #    return None, None
    page = requests.get(url, headers=headers)
    #print(page.status_code)
    #if page.status_code == :
    #    print('You got blocked, ', page.status_code)
    soup1 = BeautifulSoup(page.content, 'html.parser')
    soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
    
    # get prodocts urls
    urls = []
    for h in soup2.find_all("a",{"class":"a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"}):
        url_string = h.attrs['href']
        sep = '/ref='
        url_string_stripped = url_string.split(sep, 1)[0]
        urls.append('https://www.amazon.com' + url_string_stripped)

    # get next page
    #print(soup2.find_all("a",{'class':"s-pagination-item s-pagination-button"}))
    # s-pagination-item s-pagination-next s-pagination-button s-pagination-separator
    next_url = soup2.find("a",{"class":'s-pagination-next'})
    if next_url != None:
        next_url = next_url.get("href")
        next_url = 'https://www.amazon.com' + next_url
    
    return next_url, urls


In [132]:
# GPU = graphics_coprocessor

In [147]:
def get_list_laptop_id_with_screenTouch(url, headers):
    t_sleep = 10 #how many sec to sleep between search pages
    lst_laptops = []
    
    # the main loop, how many pages to scrap
    for i in range(1, 3):
        #print('search page: ', i)
        # scrap the search page i (returns the url of next page and prodocts urls)
        next_url, prodocts_urls = search_page_handler(url, headers)
        
        # going over the urls of the prodocts
        for url_prodoct in prodocts_urls:
            #print(url_prodoct)
            # making sure not to call a none url
            if(url_prodoct != None):
                # gets ID 
                result = re.search('\w*dp/(.*)', url_prodoct)
                if(result):
                    lst_laptops.append(result.group(1))
        
        url = next_url
        if next_url == None:
            break
        time.sleep(t_sleep) # in secs
    
    return lst_laptops

In [173]:
def add_col_trait(url, headers, df, name_col):
    lst_touch = get_list_laptop_id_with_screenTouch(url_touch_bar, headers)
    
    lst_touch_col = [0] * len(df)
    for pro_id in lst_touch:
        if pro_id in df['ID']:
            index_val = df[df['ID']== pro_id].index.values
            lst_touch_col[index_val] = 1
    
    # add list to df as col
    df[name_col] = lst_touch_col
    
    return df

In [172]:
def scrap_prodocts_from_urls_lst(lst_urls, headers, csv_name):
    list_info = []
    # going over the urls of the prodocts
    for url_prodoct in lst_urls:
        if(url_prodoct != None):
            # gets info of the prodocts and returns it in a list
            list_info = single_prodocts_handle(url_prodoct, headers)
            
            # get ID
            result = re.search('\w*dp/(.*)', url_prodoct)
            if(result):
                list_info.insert(0, result.group(1))
            
            # put prodoct in csv as new row
            # Open our existing CSV file in append mode
            with open(csv_name, 'a', encoding='UTF8') as f_object:
                # Pass this file object to csv.writer() and get a writer object
                writer_obj = writer(f_object)
                # Pass the list as an argument into the writerow()
                writer_obj.writerow(list_info)
                # Close the file object
                f_object.close()
            
            list_info = []

SyntaxError: invalid non-printable character U+00A0 (94422239.py, line 18)

In [ ]:
def put_prodoct_urls_in_csv_from_search_pages(url, headers, csv_name):
    t_sleep = 10 #how many sec to sleep between search pages
    
    for i in range(1, 2):
        # scrap the search page i (returns the url of next page and prodocts urls)
        next_url, prodocts_urls = search_page_handler(url, headers)
        
        # put urls in csv
        
        
        url = next_url
        if next_url == None:
            break
        time.sleep(t_sleep) # in secs

In [157]:
def mainfunction(url, headers):
    t_sleep = 10 #how many sec to sleep between search pages
    lst_columns = ['ID','Name','Price($)','Release_Date','Brand','Model','Display.size(Inches)','Hard_Disk.size(GB)','CPU','RAM_size(GB)','Operating_system','GPU']#,'TouchScreen']
    df = pd.DataFrame([], columns = lst_columns)

    # the main loop, how many pages to scrap
    list_info = []
    for i in range(1, 2):
        #print('search page: ', i)
        # scrap the search page i (returns the url of next page and prodocts urls)
        next_url, prodocts_urls = search_page_handler(url, headers)

        # going over the urls of the prodocts
        for url_prodoct in prodocts_urls:
            #print(url_prodoct)
            # making sure not to call a none url
            if(url_prodoct != None):
                # gets info of the prodocts and returns it in a list
                list_info = single_prodocts_handle(url_prodoct, headers)
                result = re.search('\w*dp/(.*)', url_prodoct)
                if(result):
                    list_info.insert(0, result.group(1))
                if(len(list_info) == len(lst_columns)):
                    df.loc[len(df)] = list_info
                #print(list_info)
                list_info = []
        
        url = next_url
        if next_url == None:
            break
        time.sleep(t_sleep) # in secs
       
    return df
    
# side note, for 5,000 prodocts, (while a page has ~35), will need 145 sreach pages.

In [164]:
# MAIN - Where we start
# create CSV:
csv_file_name = 'test_1.csv'
#df.to_csv(csv_file_name)

# what will be use
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:108.0) Gecko/20100101 Firefox/108.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
#headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

# the starting point (should be a search page og laptops in amazon)
url_0 = 'https://www.amazon.com/s?k=Laptop&i=electronics&bbn=172282&rh=n%3A172282%2Cp_89%3AASUS%7CApple%7CDell%7CHP%7CLenovo&dc&ds=v1%3AZaooOyrmACgMuUriyaxCaaS3MTydkF4mcXIYLENdXOQ&crid=36LMVV4MB69KG&qid=1671818059&rnid=2528832011&sprefix=laptop%2Caps%2C248&ref=sr_nr_p_89_9'
# start the projoct
df = mainfunction(url_0, headers)

print(df.head)

url_touch_bar = 'https://www.amazon.com/s?k=Laptop&i=electronics&bbn=172282&rh=n%3A565108%2Cp_89%3AASUS%7CApple%7CDell%7CHP%7CLenovo%2Cp_n_feature_fifteen_browse-bin%3A23936568011&dc&ds=v1%3AVnaR7kkl6HGSbLEeS1405RlfMGP7V6%2BRiK0qmHEGCM4&crid=36LMVV4MB69KG&qid=1673705655&rnid=23611390011&sprefix=laptop%2Caps%2C248&ref=sr_nr_p_n_feature_fifteen_browse-bin_1'
url_touch_screen = 'https://www.amazon.com/s?k=Laptop&i=electronics&bbn=172282&rh=n%3A565108%2Cp_89%3AASUS%7CApple%7CDell%7CHP%7CLenovo%2Cp_n_feature_fifteen_browse-bin%3A23611396011&dc&ds=v1%3ARGJ%2BVa61Yq9GURz9sQ%2BDyXYRhwE899lNYh%2FghT7u%2FoQ&crid=36LMVV4MB69KG&qid=1673705655&rnid=23611390011&sprefix=laptop%2Caps%2C248&ref=sr_nr_p_n_feature_fifteen_browse-bin_2'
url_touch_screen_with_stylus_support = 'https://www.amazon.com/s?k=Laptop&i=electronics&bbn=172282&rh=n%3A565108%2Cp_89%3AASUS%7CApple%7CDell%7CHP%7CLenovo%2Cp_n_feature_fifteen_browse-bin%3A23611397011&dc&ds=v1%3ADnd31uHM0rart9ssD%2Fmazk6C6X97pdWbJ5OcpaIK3nE&crid=36LMVV4MB69KG&qid=1673705655&rnid=23611390011&sprefix=laptop%2Caps%2C248&ref=sr_nr_p_n_feature_fifteen_browse-bin_3'

df = add_col_trait(url_touch_bar, headers, df, 'Touch_Bar')
print(df['Touch_Bar'].unique())
df = add_col_trait(url_touch_screen, headers, df, 'touch_screen')
print(df['touch_screen'].unique())
df = add_col_trait(url_touch_screen_with_stylus_support, headers, df, 'touch_screen_and_stylus')
print(df['touch_screen_and_stylus'].unique())

print(df.head)


200
next is: <a aria-label="Go to next page, page 2" class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator" href="/s?k=Laptop&amp;i=electronics&amp;bbn=172282&amp;rh=n%3A172282%2Cp_89%3AASUS%7CApple%7CDell%7CHP%7CLenovo&amp;dc&amp;page=2&amp;crid=36LMVV4MB69KG&amp;qid=1673716450&amp;rnid=2528832011&amp;sprefix=laptop%2Caps%2C248&amp;ref=sr_pg_1">
              Next
              <svg aria-hidden="true" focusable="false" height="12" viewbox="0 0 8 12" width="8" xmlns="http://www.w3.org/2000/svg">
<path d="M2.126.35a1.28 1.28 0 00-1.761 0 1.165 1.165 0 000 1.695L4.478 6 .365 9.955a1.165 1.165 0 000 1.694 1.28 1.28 0 001.76 0L8 6 2.126.35z">
</path>
</svg>
</a>
next is: https://www.amazon.com/s?k=Laptop&i=electronics&bbn=172282&rh=n%3A172282%2Cp_89%3AASUS%7CApple%7CDell%7CHP%7CLenovo&dc&page=2&crid=36LMVV4MB69KG&qid=1673716450&rnid=2528832011&sprefix=laptop%2Caps%2C248&ref=sr_pg_1


KeyboardInterrupt: 

In [21]:
# csv file code
'''
# in mainfunction func
# CREATE NEW CSV
    csv_name = 'Test_1.csv'
    lst_columns = ['Number','Name','Price','Year_relise','Brand','Model','Display.size',
    'Hard_Disk.size','CPU','RAM_size','Operating_system','GPU']
    with open(csv_name, 'w', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(lst_columns)
        f.close()
# ADD TO CSV
                # Open our existing CSV file in append mode
                with open(csv_name, 'a', encoding='UTF8') as f_object:
                    # Pass this file object to csv.writer() and get a writer object
                    writer_object = writer(f_object)
                    # Pass the list as an argument into the writerow()
                    writer_object.writerow(list_info)
                    # Close the file object
                    f_object.close()
'''


"\n# in mainfunction func\n# CREATE NEW CSV\n    csv_name = 'Test_1.csv'\n    lst_columns = ['Number','Name','Price','Year_relise','Brand','Model','Display.size',\n    'Hard_Disk.size','CPU','RAM_size','Operating_system','GPU']\n    with open(csv_name, 'w', encoding='UTF8') as f:\n        writer = csv.writer(f)\n        writer.writerow(lst_columns)\n        f.close()\n# ADD TO CSV\n                # Open our existing CSV file in append mode\n                with open(csv_name, 'a', encoding='UTF8') as f_object:\n                \xa0\xa0\xa0\xa0# Pass this file object to csv.writer() and get a writer object\n                \xa0\xa0\xa0\xa0writer_object = writer(f_object)\n                \xa0\xa0\xa0\xa0# Pass the list as an argument into the writerow()\n                \xa0\xa0\xa0\xa0writer_object.writerow(list_info)\n                \xa0\xa0\xa0\xa0# Close the file object\n                \xa0\xa0\xa0\xa0f_object.close()\n"

In [108]:
# my own: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:108.0) Gecko/20100101 Firefox/108.0
# 
# old_headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:108.0) Gecko/20100101 Firefox/108.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
url_0 = 'https://www.amazon.com/s?k=Laptop&i=electronics&bbn=172282&rh=n%3A172282%2Cp_89%3AASUS%7CApple%7CDell%7CHP%7CLenovo&dc&ds=v1%3AZaooOyrmACgMuUriyaxCaaS3MTydkF4mcXIYLENdXOQ&crid=36LMVV4MB69KG&qid=1671818059&rnid=2528832011&sprefix=laptop%2Caps%2C248&ref=sr_nr_p_89_9'
next_page = search_page_handler(url_0, headers)
print(next_page)
# delete everything starting with '/ref='


https://www.amazon.com/s?k=Laptop&i=electronics&bbn=172282&rh=n%3A172282%2Cp_89%3AASUS%7CApple%7CDell%7CHP%7CLenovo&dc&page=2&crid=36LMVV4MB69KG&qid=1673097402&rnid=2528832011&sprefix=laptop%2Caps%2C248&ref=sr_pg_1
https://www.amazon.com/s?k=Laptop&i=electronics&bbn=172282&rh=n%3A172282%2Cp_89%3AASUS%7CApple%7CDell%7CHP%7CLenovo&dc&page=2&crid=36LMVV4MB69KG&qid=1673097402&rnid=2528832011&sprefix=laptop%2Caps%2C248&ref=sr_pg_1


In [104]:
# cheaking the page_prodoct func
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:108.0) Gecko/20100101 Firefox/108.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
url_1 = "https://www.amazon.com/Intel-Celeron-N4020-14-Inch-Laptop/dp/B09YRY6QCX/ref=sr_1_11?crid=36LMVV4MB69KG&keywords=Laptop&qid=1673019112&refinements=p_89%3AASUS%7CApple%7CDell%7CHP%7CLenovo&rnid=2528832011&s=electronics&sprefix=laptop%2Caps%2C248&sr=1-11"
url_2 = "https://www.amazon.com/HP-Generation-Processor-Fingerprint-15-eg2025nr/dp/B09T513YCJ/ref=sr_1_10?crid=36LMVV4MB69KG&keywords=Laptop&qid=1673019112&refinements=p_89%3AASUS%7CApple%7CDell%7CHP%7CLenovo&rnid=2528832011&s=electronics&sprefix=laptop%2Caps%2C248&sr=1-10"
print(single_prodocts_handle(url_1, headers))
print('----')
print(single_prodocts_handle(url_2, headers))


['ASUS E410 Intel Celeron N4020 4GB 64GB 14-Inch HD LED Win 10 Laptop (Star Black)', '$129.99', 'April 25, 2022', 'ASUS', 'Asus', '14 Inches', '64 GB', 'Celeron', '4 GB', 'Windows 10', None]
----
['HP Pavilion 15 Laptop, 12th Generation Intel Core i7-1255U Processor, 16 GB RAM, 512 GB SSD, 15.6" Full HD Display, Windows 11 Pro, Fingerprint Reader, Wi-Fi & Bluetooth, HD Webcam (15-eg2025nr 2022)', '$680.00', 'April 10, 2022', 'HP', 'HP Pavilion Laptop 15-eg2025nr', '15.6 Inches', '512 GB', 'Core i7', '16 GB', 'Windows 11 Pro', None]


In [9]:
# html place link for prodoct in search page
#<a class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" 
#href="/Lenovo-Ideapad-Touchscreen-i3-1005G1-Processor/dp/B08B6F1NNR/ref=sr_1_3?crid=38LXN43XVKEAN&amp;keywords=laptop&amp;qid=1671894071&amp;refinements=p_89%3AASUS%7CApple%7CDell%7CHP%7CLenovo&amp;rnid=2528832011&amp;s=electronics&amp;sprefix=lapto%2Caps%2C212&amp;sr=1-3">
#<span class="a-size-medium a-color-base a-text-normal">Lenovo 2022 Newest Ideapad 3 Laptop, 15.6" HD Touchscreen, 11th Gen Intel Core i3-1115G4 Processor, 8GB DDR4 RAM, 256GB PCIe NVMe SSD, HDMI, Webcam, Wi-Fi 5, Bluetooth, Windows 11 Home, Almond
#</span> </a>

In [10]:
# html for next page
# <a href="/s?k=laptop&amp;i=electronics&amp;bbn=172282&amp;rh=n%3A172282%2Cp_89%3AASUS%7CApple%7CDell%7CHP%7CLenovo&amp;dc&amp;page=2&amp;crid=38LXN43XVKEAN&amp;qid=1671894295&amp;rnid=2528832011&amp;sprefix=lapto%2Caps%2C212&amp;ref=sr_pg_2" 
# aria-label="Go to page 2" class="s-pagination-item s-pagination-button">
# 2</a>

In [52]:
# temp for testing the temps
URL = 'https://www.amazon.com/Apple-MacBook-16-Inch-Storage-2-6GHz/dp/B08CZT64VP/ref=sr_1_1?crid=38LXN43XVKEAN&keywords=laptop&qid=1671812796&refinements=p_89%3AApple&rnid=2528832011&s=electronics&sprefix=lapto%2Caps%2C212&sr=1-1'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

page = requests.get(URL, headers=headers)
soup1 = BeautifulSoup(page.content, 'html.parser')
soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

In [67]:
# temp find price
#<span class="a-offscreen">$951.48</span>
#price = soup2.find_all("span", {"class": "a-offscreen"})

#<span class="a-price-whole">218<span class="a-price-decimal">.</span></span>
price = soup2.find_all("span", {"class": "a-price-whole"})
#print(price)

#a-price
price = soup2.find_all("span", {"class": "a-price"})
#print(price)

#apexPriceToPay
price = soup2.find_all("span", {"class": "apexPriceToPay"})
for ext in price:
    place_name = ext.find("span", {"class": "a-offscreen"}).getText().strip()
    print(place_name)
#PriceToPay
price = soup2.find_all("span", {"class": "PriceToPay"})
for ext in price:
    place_name = ext.find("span", {"class": "a-offscreen"}).getText().strip()
    print(place_name)

$955.50


In [17]:
#<td class="a-size-base prodDetAttrValue"> B08B6F1NNR </td>
#year = soup2.find_all("td", {"class": "a-size-base prodDetAttrValue"})
#print(year)

In [42]:
# temp find year
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

for td in soup2.find_all("td", {"class": "a-size-base prodDetAttrValue"}):
    if any(ext in td.text for ext in months):
        year = td.text.strip()
        print(year)

July 15, 2020


In [48]:
# temp 1 finding traits
place_trait = soup2.find_all("tr", {"class": "po-brand"})
for ext in place_trait:
    place_name = ext.find("td", {"class": "a-span9"}).getText().strip()
    print(place_name)

place_trait = soup2.find_all("tr", {"class": "po-model_name"})
for ext in place_trait:
    place_name = ext.find("td", {"class": "a-span9"}).getText().strip()
    print(place_name)

place_trait = soup2.find_all("tr", {"class": "po-display.size"})
for ext in place_trait:
    place_name = ext.find("td", {"class": "a-span9"}).getText().strip()
    print(place_name)
#po-hard_disk.size
place_trait = soup2.find_all("tr", {"class": "po-hard_disk.size"})
for ext in place_trait:
    place_name = ext.find("td", {"class": "a-span9"}).getText().strip()
    print(place_name)
#po-cpu_model.family
place_trait = soup2.find_all("tr", {"class": "po-cpu_model.family"})
for ext in place_trait:
    place_name = ext.find("td", {"class": "a-span9"}).getText().strip()
    print(place_name)
#po-ram_memory.installed_size
place_trait = soup2.find_all("tr", {"class": "po-ram_memory.installed_size"})
for ext in place_trait:
    place_name = ext.find("td", {"class": "a-span9"}).getText().strip()
    print(place_name)
#po-operating_system
place_trait = soup2.find_all("tr", {"class": "po-operating_system"})
for ext in place_trait:
    place_name = ext.find("td", {"class": "a-span9"}).getText().strip()
    print(place_name)
#po-graphics_coprocessor
place_trait = soup2.find_all("tr", {"class": "po-graphics_coprocessor"})
for ext in place_trait:
    place_name = ext.find("td", {"class": "a-span9"}).getText().strip()
    print(place_name)

Apple
MacBook Pro 16-inch
16 Inches
512 GB
Core i7
16 GB
Mac OS X 10.0 Cheetah
Intel UHD Graphics 630


In [76]:
# temp 2 finding traits (shorter)
traiets = ['po-brand','po-model_name','po-display.size','po-hard_disk.size','po-cpu_model.family','po-ram_memory.installed_size','po-operating_system','po-graphics_coprocessor']
print(len(traiets))
for trait in traiets:
    place_trait = soup2.find_all("tr", {"class": trait})
    for ext in place_trait:
        place_name = ext.find("td", {"class": "a-span9"}).getText().strip()
        print(trait,': ', place_name)
        

8
po-brand :  Apple
po-model_name :  MacBook Pro 16-inch
po-display.size :  16 Inches
po-hard_disk.size :  512 GB
po-cpu_model.family :  Core i7
po-ram_memory.installed_size :  16 GB
po-operating_system :  Mac OS X 10.0 Cheetah
po-graphics_coprocessor :  Intel UHD Graphics 630


In [ ]:
# how to do toush screen col
'''
I will srap the id number from the url of the prodocts
and will also do so for page_search with touchscreen ON
will use the id to know which ones have touchscreen

the id is: 
https://www.amazon.com/Lenovo-Ideapad-Touchscreen-i3-1005G1-Processor/dp/
B08B6F1NNR
/ref=(...)

between dp/ (id) /ref=

I should get the id outside of page_scrap func
so the second search_page scraping is faster
'''

In [67]:
# get screentouch with scraping the prodtoct page
'''#print(soup2)
#table_com = soup2.find_all("table", {"id": "HLCXComparisonTable"})
table_com = soup2.find_all("table", {"class": "a-bordered a-horizontal-stripes a-spacing-none a-size-base comparison_table"})
print(table_com)
th_val = soup2.find_all("th", {"class":'a-span3 comparison_attribute_name_column comparison_table_first_col'})
print(th_val)
th_val = soup2.find_all("tr", {"class":'comparison_other_attribute_row'})
print(th_val)
for tr_val in table_com:
    print(tr_val)
    th_val = tr_val.find("th", {"class":'a-span3 comparison_attribute_name_column comparison_table_first_col'}).getText().strip()
    print(th_val)
    th_text = th_val.find("span", {"class": 'a-size-base a-color-base'}).getText().strip()
    print(th_text)
    if(th_text == 'Human Interface Input'):
        val = tr_val.find("span", {"class": "a-span9"}).getText().strip()
        print(val)'''

[]
[]
[]


In [89]:
s = 'https://www.amazon.com/Lenovo-Ideapad-Touchscreen-i3-1005G1-Processor/dp/B08B6F1NNR'
result = re.search('\w*dp/(.*)', s)
print(result.group(1))

B08B6F1NNR
